In [1]:
from enum import IntEnum

from pydantic import Field

from spycekit import DesignBase, FeatureSpace, Population

In [2]:
import json

class Style(IntEnum):
    Cool = 0
    Fast = 1

class MyDesign(DesignBase):
    length: float = Field(..., ge=0.5, le=2.5)
    width: float = Field(..., ge=3.5, le=4.5)
    style: Style

d = MyDesign(length=0.5, width=4, style=Style.Cool)
sim = MyDesign.sample_once()
print(sim.to_series())

# Similar, but the latter can be used with datamodel-codegen
# Option 1: Convert the Design to a FeatureSpace, then dump the feature space
featurespace = MyDesign.to_featurespace()
serialized_featurespace = featurespace.model_dump_json(indent=4)
print(serialized_featurespace)

# Option 2:
# Dump the JSON Schema of the design
serialized_model_schema = json.dumps(MyDesign.model_json_schema(), indent=4)
print(serialized_model_schema)

# Deserialization
# Option 1
# Reconstruct a class which is identical to the design (up to validation) from a feature space representation!
# Assume you have already loaded the featurespace using standard pydantic deserializer
# NB: Option 2 would be using json schema and data model codegen
NewModel = featurespace.to_designmodel()
d = NewModel(length=1, width=4, style=Style.Cool)
d.style = Style.Fast
print(NewModel.sample_once())


id        b14a085c-f232-4f6e-a4f1-f2c6594f507a
length                                2.240927
width                                 4.301874
style                               Style.Cool
dtype: object
{
    "name": "MyDesign",
    "id": "da8fb83e-cc78-4307-8ae5-ec7493f976b3",
    "features": {
        "length": {
            "fieldname": "length",
            "bounds": [
                0.5,
                2.5
            ],
            "mode": "Continuous",
            "min": 0.5,
            "max": 2.5
        },
        "width": {
            "fieldname": "width",
            "bounds": [
                3.5,
                4.5
            ],
            "mode": "Continuous",
            "min": 3.5,
            "max": 4.5
        },
        "style": {
            "fieldname": "style",
            "bounds": {
                "Cool": 0,
                "Fast": 1
            },
            "mode": "Categorical",
            "min": null,
            "max": null
        }
    }
}
{
   

In [41]:
df = Population.from_feature_space(featurespace,n=0, index_by_metadata=True).data
import pandas as pd
df.loc[("al","b"),:] = [0,1,2]
df

group       features            
field         length width style
space_id id                     
al       b       0.0   1.0   2.0

In [14]:
import pandas as pd
pop = featurespace.make_population(5, index_by_metadata=False)

# df = pd.DataFrame(data={"a": range(5)},index=pd.MultiIndex.from_frame(pop.data) )
df = pd.DataFrame(index=pd.MultiIndex.from_frame(pop.data).names)

df, pop.data.columns.names, pop.data.columns.get_level_values(0), pop.data.columns.get_level_values(1)
# pop.data.columns = pop.data.columns.to_flat_index()

# Transform to a multiindexable series
df2 = pop.data.set_index(pop.data["metadata"]["id"])
df2.unstack()["features"]["length"]

# From  hierarchical to flat columns
pop.data.columns = pop.data.columns.to_flat_index()
pop.data

# from flat to heierarchical
pop.data.columns = pd.MultiIndex.from_tuples(pop.data.columns, names=["group","field"])


df = pop.data.set_index([("metadata","space_id"),("metadata","id")])


# print(pop.data.columns.to_flat_index())

# Metadata df = pd.DataFrame(index=pop.data.columns)

# import pandas as pd
# pd.DataFrame(data={"a": range(5)}, index=pd.MultiIndex.from_frame(pop.data))

# pop.data.reset_index(level="Label", names=("metadata","space_id"))

group                                                                            features  \
field                                                                              length   
(metadata, space_id)                    (metadata, id)                                      
290517938470675618075255113245515151027 182671076918680374034869685552947662223  1.042530   
                                        85356800373463795566448449910268079589   0.909280   
                                        212656097656733472273140491868840272154  0.847764   
                                        154990671846313946732284640716691465965  1.927589   
                                        69484431114681991924306424400287148706   0.891419   

group                                                                                      \
field                                                                               width   
(metadata, space_id)                    (metadata, id)                                      
290517938470675618075255113245515151027 182671076918680374034869685552947662223  3.590545   
                                        85356800373463795566448449910268079589   3.548912   
                                        212656097656733472273140491868840272154  4.357612   
                                        154990671846313946732284640716691465965  3.896172   
                                        69484431114681991924306424400287148706   4.309041   

group                                                                                  
field                                                                           style  
(metadata, space_id)                    (metadata, id)                                 
290517938470675618075255113245515151027 182671076918680374034869685552947662223     1  
                                        85356800373463795566448449910268079589      0  
                                        212656097656733472273140491868840272154     1  
                                        154990671846313946732284640716691465965     1  
                                        69484431114681991924306424400287148706      1

In [22]:
df.xs(key="width", axis=1,level=1)

group                                                                            features
(metadata, space_id)                    (metadata, id)                                   
290517938470675618075255113245515151027 182671076918680374034869685552947662223  3.590545
                                        85356800373463795566448449910268079589   3.548912
                                        212656097656733472273140491868840272154  4.357612
                                        154990671846313946732284640716691465965  3.896172
                                        69484431114681991924306424400287148706   4.309041